<a href="https://colab.research.google.com/github/eleazarpsmr/notebook-colab/blob/main/Model_EPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install openpyxl

In [3]:
from google.colab import files
import pandas as pd

# Upload file
uploaded = files.upload()

# Baca file Excel
df = pd.read_excel('XAUUSD_1m_2024.xlsx', engine='openpyxl')  # Ganti nama file sesuai yang Anda upload
trades = pd.read_excel('trade_history_2024.xlsx', engine='openpyxl')

Saving trade_history_2024.xlsx to trade_history_2024.xlsx
Saving XAUUSD_1m_2024.xlsx to XAUUSD_1m_2024.xlsx


In [4]:
# Cek 5 baris pertama data harga
print(df.head())

# Cek 5 baris pertama riwayat trade
print(trades.head())

            Timestamp     Open     High      Low    Close  Volume
0 2024-01-02 01:00:00  2062.91  2064.79  2062.80  2064.02      62
1 2024-01-02 01:01:00  2063.85  2064.46  2063.79  2064.45      39
2 2024-01-02 01:02:00  2064.45  2064.50  2064.24  2064.28      33
3 2024-01-02 01:03:00  2064.30  2064.55  2064.28  2064.30      35
4 2024-01-02 01:04:00  2064.30  2065.51  2064.24  2065.37      53
         Entry Time         Exit Time Position (Buy/Sell)  Entry Price  \
0  2024.01.04 17:59  2024.01.04 19:03                 Buy      2045.05   
1  2024.01.05 17:24  2024.01.05 17:30                 Buy      2062.22   
2  2024.01.12 17:37  2024.01.12 17:43                Sell      2054.59   
3  2024.01.17 17:16  2024.01.17 17:31                Sell      2011.00   
4  2024.01.18 16:17  2024.01.18 16:21                Sell      2010.29   

   Exit Price  Profit/Loss  
0     2043.40      -386.10  
1     2058.56      -376.98  
2     2052.07       425.88  
3     2007.94       428.40  
4     2012.13 

In [5]:
import numpy as np
import pandas as pd
from google.colab import files